# 机器学习练习 4 - 神经网络

In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from scipy.io import loadmat
from sklearn.preprocessing import OneHotEncoder
from scipy.optimize import minimize
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris,load_digits
from sklearn.decomposition import PCA
import os

In [32]:
np.random.seed()
data = loadmat('ex4data1.mat')
X = data['X']
y = data['y']
X = pd.DataFrame(X)
y = pd.DataFrame(y)
X = X.sample(n=500, replace=False, random_state=1, axis=0)
y = y.sample(n=500, replace=False, random_state=1, axis=0)
hidden_num = 25
alpha = 1

In [45]:
def plot_100_image(X):
    """ sample 100 image and show them
    assume the image is square

    X : (5000, 400)
    """
    size = int(np.sqrt(X.shape[1]))
    # sample 100 image, reshape, reorg it
    sample_idx = np.random.choice(np.arange(X.shape[0]), 100)  # 100*400
    sample_images = X[sample_idx, :]

    fig, ax_array = plt.subplots(nrows=10, ncols=10, sharey=True, sharex=True, figsize=(8, 8))

    for r in range(10):
        for c in range(10):
            ax_array[r, c].matshow(sample_images[10 * r + c].reshape((size, size)),
                                   cmap=matplotlib.cm.binary)
            plt.xticks(np.array([]))
            plt.yticks(np.array([]))


In [46]:
plot_100_image(X)
plt.show()

KeyError: (array([252, 158, 480, 253, 402,  43, 120, 462, 214, 105, 370, 427, 396,
       127, 471, 469, 321, 233, 180, 298, 289, 481, 168,  28, 209, 186,
       416, 261, 384, 169, 332, 491, 123,   7, 335, 284, 254,   1,  23,
        58,  24, 134, 181, 421,  32, 311, 289, 173, 369, 377, 134, 342,
       387, 307, 403, 284, 141,   3, 278, 322, 217, 392, 386, 177, 375,
        96, 247, 155,  18, 389, 164, 236, 180, 367, 343, 282,  78, 365,
       378,  89, 224, 227, 400, 439,  32,  31, 331, 255, 191, 437, 373,
       284, 299, 456, 370, 264, 408, 397, 351, 137]), slice(None, None, None))

In [5]:
encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y = y_onehot  
y.shape

(500, 10)

In [6]:
theta = (np.random.random(size = hidden_num * (X.shape[1] + 1) + y.shape[1] * (hidden_num + 1)) - 0.5) * 0.25

In [7]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]
    
    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    a3 = sigmoid(z3)
    
    return a1, z2, a2, z3, a3

# Backpropagation

In [8]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))
def sigmoidDerivative(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))
    

In [9]:
def cost(theta, X, y, hidden_num, alpha):
    
    '''
    Matrixing
    '''
    m = len(X)
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    '''
    Expanding theta
    '''
    theta1 = np.reshape(theta[:, :hidden_num * (X.shape[1] + 1)], (hidden_num, X.shape[1] + 1)) #(25, 401)
    theta2 = np.reshape(theta[:, hidden_num * (X.shape[1] + 1):], (y.shape[1] , (hidden_num + 1)))#(10, 26)

    '''
    Forward Propagation
    '''
    z1 = X #(500, 400)
    a1 = np.insert(z1, 0, values=np.ones(X.shape[0]), axis=1)#(500, 401)
    z2 = a1 * theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, values=np.ones(X.shape[0]), axis=1)#(500, 26)
    z3 = a2 * theta2.T
    a3 = sigmoid(z3) #(500, 10)
    '''
    Cost Function
    '''
    J = 0
    for i in range(len(X)):
        first_term = np.multiply(-y[i,:], np.log(a3[i,:]))
        second_term = np.multiply((1 - y[i,:]), np.log(1 - a3[i,:]))
        J += np.sum(first_term - second_term)
    
    J = J / len(X)
    return J

In [10]:
def BP(theta, X, y, hidden_num, alpha):

    '''
    Matrixing
    '''
    m = len(X)
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    '''
    Expanding theta
    '''
    theta1 = np.reshape(theta[:, :hidden_num * (X.shape[1] + 1)], (hidden_num, X.shape[1] + 1)) #(25, 401)
    theta2 = np.reshape(theta[:, hidden_num * (X.shape[1] + 1):], (y.shape[1] , (hidden_num + 1)))#(10, 26)


    '''
    Forward Propagation
    '''
    z1 = X #(500, 400)
    a1 = np.insert(z1, 0, values=np.ones(X.shape[0]), axis=1)#(500, 401)
    z2 = a1 * theta1.T
    a2 = sigmoid(z2)
    a2 = np.insert(a2, 0, values=np.ones(X.shape[0]), axis=1)#(500, 26)
    z3 = a2 * theta2.T
    a3 = sigmoid(z3) #(500, 10)
    # '''
    # Cost Function
    # '''
    # # cos =np.multiply(-y, np.log(sigmoid(a3))) - np.multiply((1 - y), np.log(1 - sigmoid(a3)))
    # # reg = np.sum(np.power(theta2, 2)) * (alpha / (2 * len(a2))) + np.sum(np.power(theta1, 2)) * (alpha / len(z1))
    # # Cost = np.sum(cos) / len(X) + reg
    # J = 0
    # for i in range(len(X)):
    #     first_term = np.multiply(-y[i,:], np.log(a3[i,:]))
    #     second_term = np.multiply((1 - y[i,:]), np.log(1 - a3[i,:]))
    #     J += np.sum(first_term - second_term)
    
    # J = J / len(X)
    
    '''
    BackPropagation
    '''
    delta1 = np.zeros((hidden_num, X.shape[1] + 1)) #(25, 401)
    delta2 = np.zeros((y.shape[1], hidden_num + 1)) #(10, 26)

    for i in range(X.shape[0]):
        y_i = y[i, :]
        a1i = a1[i, :]
        a2i = a2[i, :]
        a3i = a3[i, :]
        z2i = np.insert(z2[i, :], 0, values=np.ones(1), axis=1) #(1, 26)

        d3 = a3i - y_i #(1, 10)
        d2 = np.multiply((theta2.T * d3.T).T, sigmoidDerivative(z2i)) #(1, 26)
        d2 = d2[:, 1:]
    

        delta1 += (d2.T * a1i)
        delta2 += (d3.T * a2i)

    delta1 = delta1 / len(X) #(25, 401) 
    delta2 = delta2 / len(X) #(10, 26)

    delta1[:,1:] = delta1[:,1:] + (theta1[:,1:] * alpha) / len(X)
    delta2[:,1:] = delta2[:,1:] + (theta2[:,1:] * alpha) / len(X)

    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2))) #25 * 401 + 10 * 26
    
    return grad



In [11]:
grad = BP(theta, X, y, hidden_num, alpha)
grad


array([5.85526343e-04, 2.04016692e-05, 5.76991816e-05, ...,
       2.51506869e-01, 2.48008860e-01, 2.80670120e-01])

In [12]:
def gradientDescent(X, y, hidden_num, theta, epochs, alpha):
    X = np.matrix(X)
    y = np.matrix(y)
    theta = np.matrix(theta)

    theta1 = np.reshape(theta[:, :hidden_num * (X.shape[1] + 1)], (hidden_num, X.shape[1] + 1)) #(25, 401)
    theta2 = np.reshape(theta[:, hidden_num * (X.shape[1] + 1):], (y.shape[1] , (hidden_num + 1)))#(10, 26)

    for i in range(epochs):
        grad = BP(theta, X, y, hidden_num, alpha)
        theta -= grad * alpha
        if i % 50 ==0:
            print(cost(theta, X, y, hidden_num, alpha))
        else:
            cost(theta, X, y, hidden_num, alpha)
   
    return theta

In [13]:
# fmin = minimize(fun=BP, x0=theta, args=(X, y, hidden_num, alpha), method='TNC', jac=True, options={'maxiter':250})
# fmin
#Cost函数出大问题
# If jac is a Boolean and is True, fun is assumed to return and objective and gradient as an (f, g) tuple.

In [14]:
theta = gradientDescent(X, y, hidden_num, theta, 500, alpha)


3.374410177810357
1.8750708986242741
1.1370689871216277
0.836335045204263
0.6741466790152109
0.5735772694524562
0.5073664632281857
0.46153245362088097
0.4285264331575242
0.4040550925863933


In [15]:
X = np.matrix(X)
theta = np.matrix(theta)

theta1 = np.matrix(np.reshape(theta[:, :hidden_num * (X.shape[1] + 1)], (hidden_num, X.shape[1] + 1)))
theta2 = np.matrix(np.reshape(theta[:, hidden_num * (X.shape[1] + 1):], (y.shape[1] , (hidden_num + 1))))
print(theta1.shape, theta2.shape)

a1, z2, a2, z3, a3 = forward_propagate(X, theta1, theta2)
y_pred = np.array(np.argmax(a3, axis=1) + 1)
y_pred.shape

(25, 401) (10, 26)


(500, 1)

In [16]:
y = pd.DataFrame(data['y'])
y = y.sample(n=500, replace=False, random_state=1, axis=0)
y = np.array(y)
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 98.8%
